# Define imports

In [10]:
#!pip install opencv-python
import os
import cv2
import os
import glob
import numpy as np
from Regressor_Model_Controller import Regressor_Model_Controller
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torch
import torchvision.transforms as transforms
import math
import random


torch.set_default_tensor_type(torch.FloatTensor)

# Define constants

In [17]:

DIPALI_HOME = './DLWG/DataSet/*.jpg'
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
REGRESSOR_SAVED_MODEL_PATH= './Model/Regressor/Regressor.pth'
REGRESSOR_EPOCH = 75
REGRESSOR_LOSS_PLOT_PATH = "./Plots/Regressor/Regressor_Loss_plot"
REGRESSOR_LR = 0.0001
REGRESSOR_WEIGHT_DECAY = 1e-5
REGRESSOR_IN_CHANNEL = 1
REGRESSOR_HIDDEN_CHANNEL = 3
REGRESSOR_OUT_DIMS = 2
REGRESSOR_BATCH_SIZE_CPU = 32
REGRESSOR_BATCH_SIZE_CUDA = 32

os.makedirs("./Model/Regressor/", exist_ok=True)
os.makedirs("./Plots/Regressor/",exist_ok=True)

# Use GPU if available

In [3]:
def get_device():
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        is_cuda_present = True if torch.cuda.is_available() else False
        num_workers = 8 if is_cuda_present else 0

        return device, is_cuda_present, num_workers

# Load image data from folder

In [4]:
def image_modification(file,nimages):
    img = cv2.imread(file)

    #Resize to respect the input_shape
    inp = cv2.resize(img, (IMAGE_WIDTH , IMAGE_HEIGHT))

    #Convert img to RGB
    rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)
    
    return img
    

def read_images_to_tensor(filepath):
    
    files = glob.glob(filepath)
    nimages = len(files)
    data = []
    #print(files)
    for f1 in files:
        #cv2.imshow('image',img)
        data.append(image_modification(f1,nimages))
    return torch.tensor(data)


data_tensor = read_images_to_tensor(DIPALI_HOME)

# Define input and output directories

In [5]:
data_set_dir = './DataSet'
image_paths = os.listdir(data_set_dir)
image_paths = [image_path for image_path in image_paths if image_path.endswith('.jpg')]
random.shuffle(image_paths)
image_data = [cv2.imread(os.path.join(data_set_dir, image_path)) for image_path in image_paths]

# Convert images to LAB color space¶

In [6]:
transformed_image_data = []

transform_color = transforms.Compose([transforms.ToTensor()])

for image_path in image_paths:
    image = cv2.imread(os.path.join(data_set_dir, image_path))
    imageLAB = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    #print(imageLAB.shape)
    image_lab = transform_color(image)
    image_gray = image_lab[0:1,:,:]

    image_a = image_lab[1:2, :, :]
    #print(image_a.shape)

    image_b = image_lab[2:3, :, :]
    #print(image_b.shape)
    #print(image_b)
    #plt.imshow(image_b)
    #plt.show()
    

    transformed_image_data.append((image_gray, image_a, image_b))


In [17]:
#transformed_image_data[:5]

[(tensor([[[0.1882, 0.1490, 0.0863,  ..., 0.0510, 0.0314, 0.0471],
           [0.1608, 0.1098, 0.0510,  ..., 0.0706, 0.0431, 0.0471],
           [0.1137, 0.0706, 0.0549,  ..., 0.0745, 0.0510, 0.0353],
           ...,
           [0.0431, 0.0471, 0.0627,  ..., 0.0627, 0.0510, 0.0588],
           [0.0431, 0.0431, 0.0510,  ..., 0.0588, 0.0471, 0.0588],
           [0.0431, 0.0549, 0.0549,  ..., 0.0471, 0.0588, 0.0510]]]),
  tensor([[[0.2000, 0.1608, 0.0941,  ..., 0.0510, 0.0314, 0.0471],
           [0.1725, 0.1216, 0.0588,  ..., 0.0706, 0.0431, 0.0471],
           [0.1137, 0.0745, 0.0588,  ..., 0.0667, 0.0510, 0.0353],
           ...,
           [0.1255, 0.1294, 0.1412,  ..., 0.1294, 0.1255, 0.1333],
           [0.1294, 0.1294, 0.1373,  ..., 0.1333, 0.1216, 0.1333],
           [0.1294, 0.1412, 0.1412,  ..., 0.1216, 0.1333, 0.1255]]]),
  tensor([[[0.2588, 0.2157, 0.1333,  ..., 0.0510, 0.0314, 0.0471],
           [0.2314, 0.1765, 0.0980,  ..., 0.0706, 0.0431, 0.0471],
           [0.1765, 0.12

In [7]:
test_size=0.1
batch_size = REGRESSOR_BATCH_SIZE_CUDA
index_test = math.floor(len(image_data) * test_size)

# include both color and gray images
trainloader = torch.utils.data.DataLoader(transformed_image_data[:index_test], batch_size=batch_size,
                                          shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(transformed_image_data[index_test:], batch_size=batch_size,
                                         shuffle=False, num_workers=0)

# Train the Regressor

In [15]:
def train_regressor(augmented_dataset_batch, device):
        regressor_train_arguments = {
            "data_loader": augmented_dataset_batch,
            "saved_model_path": REGRESSOR_SAVED_MODEL_PATH,
            "epochs": REGRESSOR_EPOCH,
            "learning_rate": REGRESSOR_LR,
            "weight_decay": REGRESSOR_WEIGHT_DECAY,
            "in_channel": REGRESSOR_IN_CHANNEL,
            "hidden_channel": REGRESSOR_HIDDEN_CHANNEL,
            "out_dims": REGRESSOR_OUT_DIMS,
            "loss_plot_path": REGRESSOR_LOSS_PLOT_PATH
        }

        regressor_manager = Regressor_Model_Controller()
        regressor_manager.train(regressor_train_arguments, device)

def test_regressor(augmented_dataset_batch, device):
        regressor_arguments = {
            "data_loader": augmented_dataset_batch,
            "saved_model_path": REGRESSOR_SAVED_MODEL_PATH,
            "in_channel": REGRESSOR_IN_CHANNEL,
            "hidden_channel": REGRESSOR_HIDDEN_CHANNEL,
            "out_dims": REGRESSOR_OUT_DIMS,
            "loss_plot_path": REGRESSOR_LOSS_PLOT_PATH
        }

        regressor_manager = Regressor_Model_Controller()
        regressor_manager.test(regressor_arguments, device)

In [19]:
device, is_cuda_present, num_workers = get_device()
train_regressor(trainloader,device)

..Regressor training started..
1 3 2
epoch: 0, loss: 0.06240634247660637
epoch: 1, loss: 0.05197205673903227
epoch: 2, loss: 0.03328515635803342
epoch: 3, loss: 0.031295583583414555
epoch: 4, loss: 0.03182175848633051
epoch: 5, loss: 0.03041766583919525
epoch: 6, loss: 0.024337287060916424
epoch: 7, loss: 0.018735202960669994
epoch: 8, loss: 0.018919976893812418
epoch: 9, loss: 0.01610909914597869
epoch: 10, loss: 0.014970219228416681
epoch: 11, loss: 0.010565378703176975
epoch: 12, loss: 0.006635320954956114
epoch: 13, loss: 0.0067693841410800815
epoch: 14, loss: 0.005442503024823964
epoch: 15, loss: 0.006160974968224764
epoch: 16, loss: 0.0036481964925769717
epoch: 17, loss: 0.0038652048096992075
epoch: 18, loss: 0.0036625814391300082
epoch: 19, loss: 0.003440808621235192
epoch: 20, loss: 0.0039158950094133615
epoch: 21, loss: 0.003516243421472609
epoch: 22, loss: 0.002887295209802687
epoch: 23, loss: 0.003483320469968021
epoch: 24, loss: 0.0029734611744061112
epoch: 25, loss: 0.0032

<Figure size 432x288 with 0 Axes>